In [1]:
!conda create -n newCondaEnvironment -c cctbx202208 -y
!source /opt/conda/bin/activate newCondaEnvironment && conda install -c cctbx202208 python=3.6.13 -y

!source activate newCondaEnvironment && python --version

import os

Retrieving notices: ...working... done
Channels:
 - cctbx202208
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
Platform: linux-64
Solving environment: - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.9.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda/envs/newCondaEnvironment



Preparing transaction: | done
Verifying transaction: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
Executing transaction: - done
#
# To activate this environment, use
#
#     $ conda activate newCondaEnvironment
#
# To deactivate an active environment, use
#
#     $ conda deactivate

Channels:
 - cctbx202208
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
Platform: linux-64
Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
    curre

In [3]:
import os

In [4]:
!cp -r /kaggle/input/anli-data/anli_data /kaggle/working/InfoBERT/ANLI

In [5]:
os.chdir('/kaggle/working/InfoBERT')

In [6]:
!source activate newCondaEnvironment && pip install -r requirements.txt

     |################################| 753.4 MB 14 kB/s               
     |################################| 1.7 MB 69.2 MB/s            
     |################################| 674 kB 53.1 MB/s            
     |################################| 20.1 MB 58.4 MB/s            
     |################################| 6.7 MB 59.2 MB/s            
     |################################| 40 kB 4.7 MB/s             
     |################################| 63 kB 1.7 MB/s             
     |################################| 190 kB 54.6 MB/s            
     |################################| 13.1 MB 55.0 MB/s            
     |################################| 25.6 MB 49.6 MB/s            
     |################################| 211 kB 63.8 MB/s            
     |################################| 9.5 MB 44.6 MB/s            
     |################################| 63 kB 1.8 MB/s             
     |################################| 86 kB 6.1 MB/s             
     |##########################

## changing anli

In [7]:
%%writefile /kaggle/working/InfoBERT/ANLI/processors/anli.py

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" GLUE processors and helpers """
import json
import logging
import os
from enum import Enum
from typing import List, Optional, Union

from sklearn.metrics import matthews_corrcoef
from transformers import is_tf_available
from transformers import PreTrainedTokenizer
from transformers import DataProcessor, InputExample, InputFeatures
from transformers.data.metrics import simple_accuracy, acc_and_f1, pearson_and_spearman

if is_tf_available():
    import tensorflow as tf

logger = logging.getLogger(__name__)


def glue_compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    if task_name == "cola":
        return {"mcc": matthews_corrcoef(labels, preds)}
    elif task_name == "sst-2":
        return {"acc": simple_accuracy(preds, labels)}
    elif task_name == "mrpc":
        return acc_and_f1(preds, labels)
    elif task_name == "sts-b":
        return pearson_and_spearman(preds, labels)
    elif task_name == "qqp":
        return acc_and_f1(preds, labels)
    elif "mnli" in task_name:
        return {"acc": simple_accuracy(preds, labels)}
    elif task_name == "qnli":
        return {"acc": simple_accuracy(preds, labels)}
    elif task_name == "rte":
        return {"acc": simple_accuracy(preds, labels)}
    elif task_name == "wnli":
        return {"acc": simple_accuracy(preds, labels)}
    elif task_name == "hans":
        return {"acc": simple_accuracy(preds, labels)}
    elif 'snli' in task_name:
        return {"acc": simple_accuracy(preds, labels)}
    elif 'anli' in task_name:
        return {"acc": simple_accuracy(preds, labels)}
    else:
        raise KeyError(task_name)


def glue_convert_examples_to_features(
    examples: Union[List[InputExample], "tf.data.Dataset"],
    tokenizer: PreTrainedTokenizer,
    max_length: Optional[int] = None,
    task=None,
    label_list=None,
    output_mode=None,
):
    """
    Loads a data file into a list of ``InputFeatures``

    Args:
        examples: List of ``InputExamples`` or ``tf.data.Dataset`` containing the examples.
        tokenizer: Instance of a tokenizer that will tokenize the examples
        max_length: Maximum example length. Defaults to the tokenizer's max_len
        task: GLUE task
        label_list: List of labels. Can be obtained from the processor using the ``processor.get_labels()`` method
        output_mode: String indicating the output mode. Either ``regression`` or ``classification``

    Returns:
        If the ``examples`` input is a ``tf.data.Dataset``, will return a ``tf.data.Dataset``
        containing the task-specific features. If the input is a list of ``InputExamples``, will return
        a list of task-specific ``InputFeatures`` which can be fed to the model.

    """
    if is_tf_available() and isinstance(examples, tf.data.Dataset):
        if task is None:
            raise ValueError("When calling glue_convert_examples_to_features from TF, the task parameter is required.")
        return _tf_glue_convert_examples_to_features(examples, tokenizer, max_length=max_length, task=task)
    return _glue_convert_examples_to_features(
        examples, tokenizer, max_length=max_length, task=task, label_list=label_list, output_mode=output_mode
    )


if is_tf_available():

    def _tf_glue_convert_examples_to_features(
        examples: tf.data.Dataset, tokenizer: PreTrainedTokenizer, task=str, max_length: Optional[int] = None,
    ) -> tf.data.Dataset:
        """
        Returns:
            A ``tf.data.Dataset`` containing the task-specific features.

        """
        processor = glue_processors[task]()
        examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
        features = glue_convert_examples_to_features(examples, tokenizer, max_length=max_length, task=task)

        def gen():
            for ex in features:
                yield (
                    {
                        "input_ids": ex.input_ids,
                        "attention_mask": ex.attention_mask,
                        "token_type_ids": ex.token_type_ids,
                    },
                    ex.label,
                )

        return tf.data.Dataset.from_generator(
            gen,
            ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
            (
                {
                    "input_ids": tf.TensorShape([None]),
                    "attention_mask": tf.TensorShape([None]),
                    "token_type_ids": tf.TensorShape([None]),
                },
                tf.TensorShape([]),
            ),
        )


def _glue_convert_examples_to_features(
    examples: List[InputExample],
    tokenizer: PreTrainedTokenizer,
    max_length: Optional[int] = None,
    task=None,
    label_list=None,
    output_mode=None,
):
    if max_length is None:
        max_length = tokenizer.max_len

    if task is not None:
        processor = glue_processors[task]()
        if label_list is None:
            label_list = processor.get_labels()
            logger.info("Using label list %s for task %s" % (label_list, task))
        if output_mode is None:
            output_mode = glue_output_modes[task]
            logger.info("Using output mode %s for task %s" % (output_mode, task))

    label_map = {label: i for i, label in enumerate(label_list)}

    def label_from_example(example: InputExample) -> Union[int, float, None]:
        if example.label is None:
            return None
        if output_mode == "classification":
            return label_map[example.label]
        elif output_mode == "regression":
            return float(example.label)
        raise KeyError(output_mode)

    labels = [label_from_example(example) for example in examples]

    batch_encoding = tokenizer.batch_encode_plus(
        [(example.text_a, example.text_b if example.text_b else ' ') for example in examples], max_length=max_length, pad_to_max_length=True,
    )  ## RoBERTa needs to ensure text_b is not empty string


    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        logger.info("*** Example ***")
        logger.info("guid: %s" % (example.guid))
        logger.info("features: %s" % features[i])

    return features


class OutputMode(Enum):
    classification = "classification"
    regression = "regression"


class MrpcProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence1"].numpy().decode("utf-8"),
            tensor_dict["sentence2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class TextFoolerProcessor(DataProcessor):
    """Processor for the TextFooler data set (GLUE version)."""\

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    @classmethod
    def _read_jsonl(cls, input_file):
        with open(input_file, "r", encoding="utf-8-sig") as f:
            lines = f.readlines()
            return [json.loads(line) for line in lines]

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, filename):
        """Creates examples for the training, dev and test sets."""
        examples = []
        data = self._read_jsonl(filename)
        for (i, line) in enumerate(data):
            guid = "%s-%s" % ("mnli-bert-tf", i)
            text_a = line['orig_preimise']
            text_b = line['adv_hypothesis']
            label = line['true_label']
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class BertMnliMProcessor(TextFoolerProcessor):
    def get_dev_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples(os.path.join(data_dir, "infobert_mnli_matched/infobert_adv_dataset.jsonl"))
        return examples


class BertMnliMMProcessor(TextFoolerProcessor):
    def get_dev_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples(os.path.join(data_dir, "infobert_mnli_mismatched/infobert_adv_dataset.jsonl"))
        return examples

class BertSnliProcessor(TextFoolerProcessor):
    def get_dev_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples(os.path.join(data_dir, "infobert_snli/infobert_adv_dataset.jsonl"))
        return examples

class RobertaMnliMProcessor(TextFoolerProcessor):
    def get_dev_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples(os.path.join(data_dir, "roberta_mnli_matched/roberta_adv_dataset.jsonl"))
        return examples


class RobertaMnliMMProcessor(TextFoolerProcessor):
    def get_dev_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples(os.path.join(data_dir, "roberta_mnli_mismatched/roberta_adv_dataset.jsonl"))
        return examples

class RobertaSnliProcessor(TextFoolerProcessor):
    def get_dev_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples(os.path.join(data_dir, "roberta_snli/roberta_adv_dataset.jsonl"))
        return examples


class AnliAllProcessor(DataProcessor):
    """Processor for the ANLI data set (GLUE version)."""
    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples("train_r1") + self._create_examples("train_r2") + self._create_examples("train_r3")
        return examples

    def get_dev_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples("dev_r1") + self._create_examples("dev_r2") + self._create_examples("dev_r3")
        return examples

    def get_test_examples(self, data_dir):
        """See base class."""
        examples = self._create_examples("test_r1") + self._create_examples("test_r2") + self._create_examples("test_r3")
        return examples

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        label_map = {
            0: "entailment",
            1: "neutral",
            2: "contradiction"
        }
        from nlp import load_dataset
        dataset = load_dataset('anli')
        for (i, data) in enumerate(dataset[set_type]):
            guid = "%s-%s" % (set_type, data['uid'])
            text_a = data['premise']
            text_b = data['hypothesis']
            # label = None if set_type.startswith("test") else label_map[data['label']]
            label = label_map[data['label']]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class AnliR1Processor(AnliAllProcessor):
    """Processor for the MultiNLI Mismatched data set (GLUE version)."""
    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples("dev_r1")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples("test_r1")


class AnliR2Processor(AnliAllProcessor):
    """Processor for the MultiNLI Mismatched data set (GLUE version)."""
    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples("dev_r2")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples("test_r2")


class AnliR3Processor(AnliAllProcessor):
    """Processor for the MultiNLI Mismatched data set (GLUE version)."""
    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples("dev_r3")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples("test_r3")


class AnliFullProcessor(AnliAllProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""
    def get_train_examples(self, data_dir):
        """See base class."""
        # examples = SnliProcessor().get_train_examples(os.path.join(data_dir, 'SNLI'))
        # examples += FeverProcessor().get_train_examples(os.path.join(data_dir, 'nli_fever'))
        # examples += MnliProcessor().get_train_examples(os.path.join(data_dir, 'MNLI'))
        examples = AnliAllProcessor().get_train_examples(data_dir)
        return examples


class AnliPartProcessor(AnliFullProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""
    def get_train_examples(self, data_dir):
        """See base class."""
        examples = SnliProcessor().get_train_examples(os.path.join(data_dir, 'SNLI'))
        examples += MnliProcessor().get_train_examples(os.path.join(data_dir, 'MNLI'))
        return examples


class SnliProcessor(DataProcessor):
    """Processor for the SNLI data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[7]
            text_b = line[8]
            label = None if set_type.startswith("test") else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class FeverProcessor(DataProcessor):
    """Processor for the FeverNLI data set (GLUE version)."""
    def _read_jsonl(self, path):
        """Reads a jsonl value file."""
        with open(path, "r") as f:
            import json
            return [json.loads(x) for x in list(f)]

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_jsonl(os.path.join(data_dir, "train_fitems.jsonl")), "train")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        label_map = {
            "SUPPORTS": "entailment",
            "NOT ENOUGH INFO": "neutral",
            "REFUTES": "contradiction"
        }
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, line['fid'])
            text_a = line['query']
            text_b = line['context']
            label = None if set_type.startswith("test") else label_map[line['label']]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class MnliProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_matched.tsv")), "dev_matched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_matched.tsv")), "test_matched")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[8]
            text_b = line[9]
            label = None if set_type.startswith("test") else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class MnliMismatchedProcessor(MnliProcessor):
    """Processor for the MultiNLI Mismatched data set (GLUE version)."""

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_mismatched.tsv")), "dev_mismatched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_mismatched.tsv")), "test_mismatched")


class ColaProcessor(DataProcessor):
    """Processor for the CoLA data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence"].numpy().decode("utf-8"),
            None,
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        test_mode = set_type == "test"
        if test_mode:
            lines = lines[1:]
        text_index = 1 if test_mode else 3
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[text_index]
            label = None if test_mode else line[1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples


class Sst2Processor(DataProcessor):
    """Processor for the SST-2 data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence"].numpy().decode("utf-8"),
            None,
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        text_index = 1 if set_type == "test" else 0
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[text_index]
            label = None if set_type == "test" else line[1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples


class StsbProcessor(DataProcessor):
    """Processor for the STS-B data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence1"].numpy().decode("utf-8"),
            tensor_dict["sentence2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return [None]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[7]
            text_b = line[8]
            label = None if set_type == "test" else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class QqpProcessor(DataProcessor):
    """Processor for the QQP data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["question1"].numpy().decode("utf-8"),
            tensor_dict["question2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        test_mode = set_type == "test"
        q1_index = 1 if test_mode else 3
        q2_index = 2 if test_mode else 4
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            try:
                text_a = line[q1_index]
                text_b = line[q2_index]
                label = None if test_mode else line[5]
            except IndexError:
                continue
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class QnliProcessor(DataProcessor):
    """Processor for the QNLI data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["question"].numpy().decode("utf-8"),
            tensor_dict["sentence"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["entailment", "not_entailment"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[1]
            text_b = line[2]
            label = None if set_type == "test" else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class RteProcessor(DataProcessor):
    """Processor for the RTE data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence1"].numpy().decode("utf-8"),
            tensor_dict["sentence2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["entailment", "not_entailment"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[1]
            text_b = line[2]
            label = None if set_type == "test" else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


class WnliProcessor(DataProcessor):
    """Processor for the WNLI data set (GLUE version)."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence1"].numpy().decode("utf-8"),
            tensor_dict["sentence2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, line[0])
            text_a = line[1]
            text_b = line[2]
            label = None if set_type == "test" else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


glue_tasks_num_labels = {
    "cola": 2,
    "mnli": 3,
    "anli-r1": 3,
    "anli-r2": 3,
    "anli-r3": 3,
    "anli-all": 3,
    "anli-full": 3,
    "anli-part": 3,
    "snli": 3,
    "snli-bert-adv": 3,
    "mnli-bert-adv": 3,
    "mnli-mm-bert-adv": 3,
    "snli-roberta-adv": 3,
    "mnli-roberta-adv": 3,
    "mnli-mm-roberta-adv": 3,
    "mrpc": 2,
    "sst-2": 2,
    "sts-b": 1,
    "qqp": 2,
    "qnli": 2,
    "rte": 2,
    "wnli": 2,
}

glue_processors = {
    "cola": ColaProcessor,
    "anli-r1": AnliR1Processor,
    "anli-r2": AnliR2Processor,
    "anli-r3": AnliR3Processor,
    "anli-all": AnliAllProcessor,
    "anli-full": AnliFullProcessor,
    "anli-part": AnliPartProcessor,
    "snli": SnliProcessor,
    "mnli": MnliProcessor,
    "mnli-mm": MnliMismatchedProcessor,
    "mrpc": MrpcProcessor,
    "sst-2": Sst2Processor,
    "sts-b": StsbProcessor,
    "qqp": QqpProcessor,
    "qnli": QnliProcessor,
    "rte": RteProcessor,
    "wnli": WnliProcessor,
    "snli-bert-adv": BertSnliProcessor,
    "mnli-bert-adv": BertMnliMProcessor,
    "mnli-mm-bert-adv": BertMnliMMProcessor,
    "snli-roberta-adv": RobertaSnliProcessor,
    "mnli-roberta-adv": RobertaMnliMProcessor,
    "mnli-mm-roberta-adv": RobertaMnliMMProcessor,
}

glue_output_modes = {
    "cola": "classification",
    "anli-r1": "classification",
    "anli-r2": "classification",
    "anli-r3": "classification",
    "anli-all": "classification",
    "anli-full": "classification",
    "anli-part": "classification",
    "snli": "classification",
    "mnli": "classification",
    "mnli-mm": "classification",
    "snli-bert-adv": "classification",
    "mnli-bert-adv": "classification",
    "mnli-mm-bert-adv": "classification",
    "snli-roberta-adv": "classification",
    "mnli-roberta-adv": "classification",
    "mnli-mm-roberta-adv": "classification",
    "mrpc": "classification",
    "sst-2": "classification",
    "sts-b": "regression",
    "qqp": "classification",
    "qnli": "classification",
    "rte": "classification",
    "wnli": "classification",
}


Overwriting /kaggle/working/InfoBERT/ANLI/processors/anli.py


## changing eval_anli_local

In [8]:
%%writefile /kaggle/working/InfoBERT/ANLI/eval_anli_local.py

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Finetuning the library models for sequence classification on GLUE (Bert, XLM, XLNet, RoBERTa, Albert, XLM-RoBERTa)."""

import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Callable, Dict, Optional

import numpy as np
import torch

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction

from MI_estimators import CLUB, CLUBv2, InfoNCE
from datasets.anli import GlueDataset, GlueDataTrainingArguments as DataTrainingArguments
from processors.anli import glue_output_modes, glue_tasks_num_labels, glue_compute_metrics
from local_robust_trainer import Trainer
from transformers import (
    HfArgumentParser,
    set_seed,
)
from advtraining_args import TrainingArguments

logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    load: Optional[str] = field(
        default=None, metadata={"help": "the path to load pretrained models"}
    )
    beta: float = field(
        default=0, metadata={"help": "the regularization term"}
    )
    version: float = field(
        default=-1, metadata={"help": "version of MI estimator"}
    )


def main():
    # See all possible arguments in src/transformers/advtraining_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    if (
            os.path.exists(training_args.output_dir)
            and os.listdir(training_args.output_dir)
            and training_args.do_train
            and not training_args.overwrite_output_dir
            and training_args.local_rank in [-1, 0]
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    root_dir = training_args.output_dir
    if not os.path.exists(root_dir) and training_args.local_rank in [-1, 0]:
        os.mkdir(root_dir)

    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
        handlers=[
            logging.FileHandler(os.path.join(training_args.output_dir, "log.txt")),
            logging.StreamHandler()
        ] if training_args.local_rank in [-1, 0] else [logging.StreamHandler()]
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    try:
        num_labels = glue_tasks_num_labels[data_args.task_name]
        output_mode = glue_output_modes[data_args.task_name]
    except KeyError:
        raise ValueError("Task not found: %s" % (data_args.task_name))

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=model_args.cache_dir,
        output_hidden_states=True
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )
    # take the embedding of the whole sentence as varaible y
    if model_args.version >= 0:
        if model_args.version == 0:
            mi_upper_estimator = CLUB(config.hidden_size * data_args.max_seq_length, config.hidden_size,
                                beta=model_args.beta).to(training_args.device)
            mi_upper_estimator.version = 0
            mi_estimator = None
        elif model_args.version == 1:
            mi_upper_estimator = CLUB(config.hidden_size, config.hidden_size, beta=model_args.beta).to(training_args.device)
            # mi_estimator = CLUB(config.hidden_size, config.hidden_size,  beta=model_args.beta)
            mi_upper_estimator.version = 1
            mi_estimator = None
        elif model_args.version == 2 or model_args.version == 3:
            mi_upper_estimator = CLUBv2(config.hidden_size, config.hidden_size, beta=model_args.beta).to(training_args.device)
            mi_upper_estimator.version = model_args.version
            mi_estimator = None
        elif model_args.version == 4:
            mi_estimator = InfoNCE(config.hidden_size, config.hidden_size).to(training_args.device)
            mi_upper_estimator = None
        elif model_args.version == 5:
            mi_estimator = InfoNCE(config.hidden_size, config.hidden_size).to(training_args.device)
            mi_upper_estimator = CLUBv2(config.hidden_size, config.hidden_size, beta=model_args.beta).to(training_args.device)
            mi_upper_estimator.version = 2
        elif model_args.version == 6:
            mi_estimator = InfoNCE(config.hidden_size, config.hidden_size).to(training_args.device)
            mi_upper_estimator = CLUBv2(config.hidden_size, config.hidden_size, beta=model_args.beta).to(training_args.device)
            mi_upper_estimator.version = 3
    else:
        mi_estimator = None
        mi_upper_estimator = None

    # Get datasets
    train_dataset = (
        GlueDataset(data_args, tokenizer=tokenizer) if training_args.do_train else None
    )
    eval_dataset = (
        GlueDataset(data_args, tokenizer=tokenizer, mode="dev")
        if training_args.do_eval
        else None
    )
    test_dataset = (
        GlueDataset(data_args, tokenizer=tokenizer, mode="test")
        if training_args.do_predict
        else None
    )

    def build_compute_metrics_fn(task_name: str) -> Callable[[EvalPrediction], Dict]:
        def compute_metrics_fn(p: EvalPrediction):
            if output_mode == "classification":
                preds = np.argmax(p.predictions, axis=1)
            elif output_mode == "regression":
                preds = np.squeeze(p.predictions)
            return glue_compute_metrics(task_name, preds, p.label_ids)

        return compute_metrics_fn

    if model_args.load is not None:
        print(model_args.load)
        model.load_state_dict(torch.load(os.path.join(model_args.load, "pytorch_model.bin"), map_location=training_args.device))
        if mi_estimator: #  and os.path.exists(os.path.join(model_args.load, "mi_estimator.bin"))
            mi_estimator.load_state_dict(torch.load(os.path.join(model_args.load, "mi_estimator.bin"), map_location=training_args.device))
        logger.info(f"Load successful from {model_args.load}")

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=build_compute_metrics_fn(data_args.task_name),
        mi_estimator=mi_estimator,
        mi_upper_estimator=mi_upper_estimator
    )
    trainer.tokenizer = tokenizer

    # Training
    if training_args.do_train:
        trainer.train_mi_only(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        if mi_estimator:
            torch.save(mi_estimator.state_dict(), os.path.join(training_args.output_dir, "mi_estimator.bin"))
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    eval_results = {}
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        # Loop to handle MNLI double evaluation (matched, mis-matched)
        eval_datasets = [eval_dataset]
        if data_args.task_name == "mnli":
            mnli_mm_data_args = dataclasses.replace(data_args, task_name="mnli-mm")
            eval_datasets.append(
                GlueDataset(mnli_mm_data_args, tokenizer=tokenizer, mode="dev")
            )

        if data_args.task_name == 'anli-full' or data_args.task_name == 'anli-part':
            eval_tasks = ["anli-r1", "anli-r2", "anli-r3"]
            for task in eval_tasks:
                if "mnli" in task:
                    task_data_dir = os.path.join(data_args.data_dir, "MNLI")
                elif "snli" == task:
                    task_data_dir = os.path.join(data_args.data_dir, "SNLI")
                else:
                    task_data_dir = data_args.data_dir
                task_data_args = dataclasses.replace(data_args, task_name=task, data_dir=task_data_dir)
                eval_datasets.append(
                    GlueDataset(task_data_args, tokenizer=tokenizer, mode="dev")
                )

        for eval_dataset in eval_datasets:
            trainer.compute_metrics = build_compute_metrics_fn(eval_dataset.args.task_name)
            eval_result = trainer.evaluate_mi(eval_dataset=eval_dataset)

            output_eval_file = os.path.join(
                training_args.output_dir, f"eval_results_{eval_dataset.args.task_name}.txt"
            )
            if trainer.is_world_master():
                with open(output_eval_file, "w") as writer:
                    logger.info("***** Eval results {} *****".format(eval_dataset.args.task_name))
                    for key, value in eval_result.items():
                        logger.info("  %s = %s", key, value)
                        writer.write("%s = %s\n" % (key, value))

            eval_results.update(eval_result)

    if training_args.do_predict:
        logging.info("*** Test ***")
        test_datasets = [test_dataset]
        if data_args.task_name == "mnli":
            mnli_mm_data_args = dataclasses.replace(data_args, task_name="mnli-mm")
            test_datasets.append(
                GlueDataset(mnli_mm_data_args, tokenizer=tokenizer, mode="test", cache_dir=model_args.cache_dir)
            )

        for test_dataset in test_datasets:
            predictions = trainer.predict(test_dataset=test_dataset).predictions
            if output_mode == "classification":
                predictions = np.argmax(predictions, axis=1)

            output_test_file = os.path.join(
                training_args.output_dir, f"test_results_{test_dataset.args.task_name}.txt"
            )
            if trainer.is_world_master():
                with open(output_test_file, "w") as writer:
                    logger.info("***** Test results {} *****".format(test_dataset.args.task_name))
                    writer.write("index\tprediction\n")
                    for index, item in enumerate(predictions):
                        if output_mode == "regression":
                            writer.write("%d\t%3.3f\n" % (index, item))
                        else:
                            item = test_dataset.get_labels()[item]
                            writer.write("%d\t%s\n" % (index, item))
    return eval_results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


if __name__ == "__main__":
    main()

Overwriting /kaggle/working/InfoBERT/ANLI/eval_anli_local.py


## changing run_anli

In [9]:
%%writefile /kaggle/working/InfoBERT/ANLI/run_anli.py

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Finetuning the library models for sequence classification on GLUE (Bert, XLM, XLNet, RoBERTa, Albert, XLM-RoBERTa)."""

import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Callable, Dict, Optional

import numpy as np
import torch

from transformers import AutoConfig, AutoTokenizer, EvalPrediction
from models.modeling_auto import AutoModelForSequenceClassification
from MI_estimators import CLUB, CLUBv2, InfoNCE
from datasets.anli import GlueDataset, GlueDataTrainingArguments as DataTrainingArguments
from processors.anli import glue_output_modes, glue_tasks_num_labels, glue_compute_metrics
from local_robust_trainer import Trainer
from transformers import (
    HfArgumentParser,
    set_seed,
)
from advtraining_args import TrainingArguments

logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    load: Optional[str] = field(
        default=None, metadata={"help": "the path to load pretrained models"}
    )
    beta: float = field(
        default=0, metadata={"help": "the regularization term"}
    )
    version: float = field(
        default=-1, metadata={"help": "version of MI estimator"}
    )


def main():
    # See all possible arguments in src/transformers/advtraining_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    if (
            os.path.exists(training_args.output_dir)
            and os.listdir(training_args.output_dir)
            and training_args.do_train
            and not training_args.overwrite_output_dir
            and training_args.local_rank in [-1, 0]
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    root_dir = training_args.output_dir
    if not os.path.exists(root_dir) and training_args.local_rank in [-1, 0]:
        os.mkdir(root_dir)

    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
        handlers=[
            logging.FileHandler(os.path.join(training_args.output_dir, "log.txt")),
            logging.StreamHandler()
        ] if training_args.local_rank in [-1, 0] else [logging.StreamHandler()]
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    try:
        num_labels = glue_tasks_num_labels[data_args.task_name]
        output_mode = glue_output_modes[data_args.task_name]
    except KeyError:
        raise ValueError("Task not found: %s" % (data_args.task_name))

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=model_args.cache_dir,
        output_hidden_states=True,
        attention_probs_dropout_prob=training_args.attention_probs_dropout_prob,
        hidden_dropout_prob=training_args.hidden_dropout_prob
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )
    # take the embedding of the whole sentence as varaible y
    if model_args.version >= 0:
        if model_args.version == 0:
            mi_upper_estimator = CLUB(config.hidden_size * data_args.max_seq_length, config.hidden_size,
                                beta=model_args.beta).to(training_args.device)
            mi_upper_estimator.version = 0
            mi_estimator = None
        elif model_args.version == 1:
            mi_upper_estimator = CLUB(config.hidden_size, config.hidden_size, beta=model_args.beta).to(training_args.device)
            # mi_estimator = CLUB(config.hidden_size, config.hidden_size,  beta=model_args.beta)
            mi_upper_estimator.version = 1
            mi_estimator = None
        elif model_args.version == 2 or model_args.version == 3:
            mi_upper_estimator = CLUBv2(config.hidden_size, config.hidden_size, beta=model_args.beta).to(training_args.device)
            mi_upper_estimator.version = model_args.version
            mi_estimator = None
        elif model_args.version == 4:
            mi_estimator = InfoNCE(config.hidden_size, config.hidden_size).to(training_args.device)
            mi_upper_estimator = None
        elif model_args.version == 5:
            mi_estimator = InfoNCE(config.hidden_size, config.hidden_size).to(training_args.device)
            mi_upper_estimator = CLUBv2(config.hidden_size, config.hidden_size, beta=model_args.beta).to(training_args.device)
            mi_upper_estimator.version = 2
        elif model_args.version == 6:
            mi_estimator = InfoNCE(config.hidden_size, config.hidden_size).to(training_args.device)
            mi_upper_estimator = CLUBv2(config.hidden_size, config.hidden_size, beta=model_args.beta).to(training_args.device)
            mi_upper_estimator.version = 3
    else:
        mi_estimator = None
        mi_upper_estimator = None

    # Get datasets
    train_dataset = (
        GlueDataset(data_args, tokenizer=tokenizer) if training_args.do_train else None
    )
    eval_dataset = (
        GlueDataset(data_args, tokenizer=tokenizer, mode="dev")
        if training_args.do_eval
        else None
    )
    test_dataset = (
        GlueDataset(data_args, tokenizer=tokenizer, mode="test")
        if training_args.do_predict
        else None
    )

    def build_compute_metrics_fn(task_name: str) -> Callable[[EvalPrediction], Dict]:
        def compute_metrics_fn(p: EvalPrediction):
            if output_mode == "classification":
                preds = np.argmax(p.predictions, axis=1)
            elif output_mode == "regression":
                preds = np.squeeze(p.predictions)
            return glue_compute_metrics(task_name, preds, p.label_ids)

        return compute_metrics_fn

    if model_args.load is not None:
        print(model_args.load)
        model.load_state_dict(torch.load(os.path.join(model_args.load, "pytorch_model.bin")))
        if mi_estimator:
            mi_estimator.load_state_dict(torch.load(os.path.join(model_args.load, "mi_estimator.bin")))
        logger.info(f"Load successful from {model_args.load}")

    if os.path.isdir(model_args.model_name_or_path):
        if mi_estimator:
            mi_estimator.load_state_dict(torch.load(os.path.join(model_args.model_name_or_path, "mi_estimator.bin")))
            logger.info(f"Load mi estimator successful from {model_args.model_name_or_path}")


    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=build_compute_metrics_fn(data_args.task_name),
        mi_estimator=mi_estimator,
        mi_upper_estimator=mi_upper_estimator
    )
    trainer.tokenizer = tokenizer

    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

            if mi_estimator:
                torch.save(mi_estimator.state_dict(), os.path.join(training_args.output_dir, "mi_estimator.bin"))

            torch.save(trainer.eval_hist, os.path.join(training_args.output_dir, 'eval_hist.bin'))

    # Evaluation
    eval_results = {}
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        # Loop to handle MNLI double evaluation (matched, mis-matched)
        eval_datasets = [eval_dataset]
        if data_args.task_name == "mnli":
            mnli_mm_data_args = dataclasses.replace(data_args, task_name="mnli-mm")
            eval_datasets.append(
                GlueDataset(mnli_mm_data_args, tokenizer=tokenizer, mode="dev")
            )
        if 'anli' in data_args.task_name:
            eval_datasets.append(
                    GlueDataset(data_args, tokenizer=tokenizer, mode="test")
            )

        if data_args.task_name == 'anli-full' or data_args.task_name == 'anli-part':
            eval_tasks = ["anli-r1", "anli-r2", "anli-r3", "mnli", "mnli-mm", "snli",
                          "mnli-bert-adv", "mnli-mm-bert-adv", "snli-bert-adv",
                          "mnli-roberta-adv", "mnli-mm-roberta-adv", "snli-roberta-adv"]
            for task in eval_tasks:
                if "mnli" in task and 'adv' not in task:
                    task_data_dir = os.path.join(data_args.data_dir, "MNLI")
                elif "snli" == task and 'adv' not in task:
                    task_data_dir = os.path.join(data_args.data_dir, "SNLI")
                else:
                    task_data_dir = data_args.data_dir
                task_data_args = dataclasses.replace(data_args, task_name=task, data_dir=task_data_dir)
                eval_datasets.append(
                    GlueDataset(task_data_args, tokenizer=tokenizer, mode="dev")
                )
                if 'anli' in task:
                    eval_datasets.append(
                        GlueDataset(task_data_args, tokenizer=tokenizer, mode="test")
                    )

        for eval_dataset in eval_datasets:
            trainer.compute_metrics = build_compute_metrics_fn(eval_dataset.args.task_name)
            eval_result = trainer.evaluate(eval_dataset=eval_dataset)
            # eval_result = trainer.evaluate_mi(eval_dataset=eval_dataset)

            output_eval_file = os.path.join(
                training_args.output_dir, f"eval_results_{eval_dataset.args.task_name}-{eval_dataset.mode}.txt"
            )
            if trainer.is_world_master():
                with open(output_eval_file, "w") as writer:
                    logger.info(f"***** Eval results {eval_dataset.args.task_name}-{eval_dataset.mode} *****")
                    for key, value in eval_result.items():
                        logger.info("  %s = %s", key, value)
                        writer.write("%s = %s\n" % (key, value))

            eval_results.update(eval_result)

        if trainer.eval_hist:
            best_eval = trainer.eval_hist[0]
            for eval in trainer.eval_hist:
                if eval['eval_acc'] > best_eval['eval_acc']:
                    best_eval = eval
            output_eval_file = os.path.join(
                training_args.output_dir, f"best_eval_results_{data_args.task_name}_.txt"
            )
            with open(output_eval_file, "w") as writer:
                logger.info("***** Best Eval results {} *****".format(data_args.task_name))
                for key, value in best_eval.items():
                    logger.info("  %s = %s", key, value)
                    writer.write("%s = %s\n" % (key, value))

            del trainer.model
            torch.cuda.empty_cache()
            # re-evaluate the best parameters
            checkpoint = os.path.join(training_args.output_dir, f"checkpoint-{best_eval['step']}")
            # trainer.model.load_state_dict(torch.load(os.path.join(checkpoint, 'pytorch_model.bin')))
            trainer.model = AutoModelForSequenceClassification.from_pretrained(checkpoint).to(training_args.device)
            logger.info(f"successfully load from {checkpoint}")

            for eval_dataset in eval_datasets:
                trainer.compute_metrics = build_compute_metrics_fn(eval_dataset.args.task_name)
                # eval_result = trainer.evaluate(eval_dataset=eval_dataset)
                eval_result = trainer.evaluate(eval_dataset=eval_dataset)

                output_eval_file = os.path.join(
                    training_args.output_dir, f"best_eval_results_{eval_dataset.args.task_name}-{eval_dataset.mode}.txt"
                )
                if trainer.is_world_master():
                    with open(output_eval_file, "w") as writer:
                        logger.info(f"***** Best Eval results {eval_dataset.args.task_name}--{eval_dataset.mode} *****")
                        for key, value in eval_result.items():
                            logger.info("  %s = %s", key, value)
                            writer.write("%s = %s\n" % (key, value))

            # # double eval to test whether there is stocahsticasty during evaluation
            # for eval_dataset in eval_datasets:
            #     trainer.compute_metrics = build_compute_metrics_fn(eval_dataset.args.task_name)
            #     # eval_result = trainer.evaluate(eval_dataset=eval_dataset)
            #     eval_result = trainer.evaluate(eval_dataset=eval_dataset)
            #
            #     output_eval_file = os.path.join(
            #         training_args.output_dir, f"best_eval_results_{eval_dataset.args.task_name}.txt"
            #     )
            #     if trainer.is_world_master():
            #         with open(output_eval_file, "w") as writer:
            #             logger.info("***** Best Eval results {} *****".format(eval_dataset.args.task_name))
            #             for key, value in eval_result.items():
            #                 logger.info("  %s = %s", key, value)
            #                 writer.write("%s = %s\n" % (key, value))

    if training_args.do_predict:
        logging.info("*** Test ***")
        test_datasets = [test_dataset]
        if data_args.task_name == "mnli":
            mnli_mm_data_args = dataclasses.replace(data_args, task_name="mnli-mm")
            test_datasets.append(
                GlueDataset(mnli_mm_data_args, tokenizer=tokenizer, mode="test", cache_dir=model_args.cache_dir)
            )

        for test_dataset in test_datasets:
            predictions = trainer.predict(test_dataset=test_dataset).predictions
            if output_mode == "classification":
                predictions = np.argmax(predictions, axis=1)

            output_test_file = os.path.join(
                training_args.output_dir, f"test_results_{test_dataset.args.task_name}.txt"
            )
            if trainer.is_world_master():
                with open(output_test_file, "w") as writer:
                    logger.info("***** Test results {} *****".format(test_dataset.args.task_name))
                    writer.write("index\tprediction\n")
                    for index, item in enumerate(predictions):
                        if output_mode == "regression":
                            writer.write("%d\t%3.3f\n" % (index, item))
                        else:
                            item = test_dataset.get_labels()[item]
                            writer.write("%d\t%s\n" % (index, item))
    return eval_results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


if __name__ == "__main__":
    main()

Overwriting /kaggle/working/InfoBERT/ANLI/run_anli.py


In [10]:
os.chdir('/kaggle/working/InfoBERT/ANLI')

## changing adv_training_args

In [11]:
%%writefile /kaggle/working/InfoBERT/ANLI/advtraining_args.py

import dataclasses
import json
import logging
from dataclasses import dataclass, field
from typing import Any, Dict, Optional, Tuple

from transformers.file_utils import cached_property, is_torch_available, torch_required


if is_torch_available():
    import torch


try:
    import torch_xla.core.xla_model as xm

    _has_tpu = True
except ImportError:
    _has_tpu = False


@torch_required
def is_tpu_available():
    return _has_tpu


logger = logging.getLogger(__name__)


@dataclass
class TrainingArguments:
    """
    TrainingArguments is the subset of the arguments we use in our example scripts
    **which relate to the training loop itself**.

    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    output_dir: str = field(
        metadata={"help": "The output directory where the model predictions and checkpoints will be written."}
    )
    overwrite_output_dir: bool = field(
        default=False,
        metadata={
            "help": (
                "Overwrite the content of the output directory."
                "Use this to continue training if output_dir points to a checkpoint directory."
            )
        },
    )

    do_train: bool = field(default=False, metadata={"help": "Whether to run training."})
    do_eval: bool = field(default=False, metadata={"help": "Whether to run eval on the dev set."})
    do_predict: bool = field(default=False, metadata={"help": "Whether to run predictions on the test set."})
    evaluate_during_training: bool = field(
        default=False, metadata={"help": "Run evaluation during training at each logging step."},
    )

    per_device_train_batch_size: int = field(
        default=8, metadata={"help": "Batch size per GPU/TPU core/CPU for training."}
    )
    per_device_eval_batch_size: int = field(
        default=8, metadata={"help": "Batch size per GPU/TPU core/CPU for evaluation."}
    )

    per_gpu_train_batch_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Deprecated, the use of `--per_device_train_batch_size` is preferred. "
            "Batch size per GPU/TPU core/CPU for training."
        },
    )
    per_gpu_eval_batch_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Deprecated, the use of `--per_device_eval_batch_size` is preferred."
            "Batch size per GPU/TPU core/CPU for evaluation."
        },
    )

    gradient_accumulation_steps: int = field(
        default=1,
        metadata={"help": "Number of updates steps to accumulate before performing a backward/update pass."},
    )

    learning_rate: float = field(default=5e-5, metadata={"help": "The initial learning rate for Adam."})
    weight_decay: float = field(default=0.0, metadata={"help": "Weight decay if we apply some."})
    adam_epsilon: float = field(default=1e-8, metadata={"help": "Epsilon for Adam optimizer."})
    max_grad_norm: float = field(default=1.0, metadata={"help": "Max gradient norm."})

    num_train_epochs: float = field(default=1.0, metadata={"help": "Total number of training epochs to perform."})
    max_steps: int = field(
        default=-1,
        metadata={"help": "If > 0: set total number of training steps to perform. Override num_train_epochs."},
    )
    warmup_steps: int = field(default=0, metadata={"help": "Linear warmup over warmup_steps."})

    logging_dir: Optional[str] = field(default=None, metadata={"help": "Tensorboard log dir."})
    logging_first_step: bool = field(default=False, metadata={"help": "Log and eval the first global_step"})
    logging_steps: int = field(default=500, metadata={"help": "Log every X updates steps."})
    save_steps: int = field(default=500, metadata={"help": "Save checkpoint every X updates steps."})
    save_total_limit: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "Limit the total amount of checkpoints."
                "Deletes the older checkpoints in the output_dir. Default is unlimited checkpoints"
            )
        },
    )
    no_cuda: bool = field(default=False, metadata={"help": "Do not use CUDA even when it is available"})
    seed: int = field(default=42, metadata={"help": "random seed for initialization"})

    fp16: bool = field(
        default=False,
        metadata={"help": "Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit"},
    )
    fp16_opt_level: str = field(
        default="O1",
        metadata={
            "help": (
                "For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
                "See details at https://nvidia.github.io/apex/amp.html"
            )
        },
    )
    local_rank: int = field(default=-1, metadata={"help": "For distributed training: local_rank"})

    tpu_num_cores: Optional[int] = field(
        default=None, metadata={"help": "TPU: Number of TPU cores (automatically passed by launcher script)"}
    )
    tpu_metrics_debug: bool = field(default=False, metadata={"help": "TPU: Whether to print debug metrics"})

    # adv args
    alpha: Optional[float] = field(default=0)
    adv_lr: Optional[float] = field(default=0)
    adv_steps: Optional[int] = field(default=1, metadata={"help": "should be at least 1"})
    adv_init_mag: Optional[float] = field(default=0)
    norm_type: Optional[str] = field(default="l2", metadata={"help": 'choices "l2", or "linf"'})
    adv_max_norm: Optional[float] = field(default=0, metadata={"help": "set to 0 to be unlimited"})
    hidden_dropout_prob: Optional[float] = field(default=0.1)
    attention_probs_dropout_prob: Optional[float] = field(default=0)
    cl: Optional[float] = field(default=0.5)
    ch: Optional[float] = field(default=0.9)

    @property
    def train_batch_size(self) -> int:
        if self.per_gpu_train_batch_size:
            logger.warning(
                "Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future "
                "version. Using `--per_device_train_batch_size` is preferred."
            )
        per_device_batch_size = self.per_gpu_train_batch_size or self.per_device_train_batch_size
        return per_device_batch_size * max(1, self.n_gpu)

    @property
    def eval_batch_size(self) -> int:
        if self.per_gpu_eval_batch_size:
            logger.warning(
                "Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future "
                "version. Using `--per_device_eval_batch_size` is preferred."
            )
        per_device_batch_size = self.per_gpu_eval_batch_size or self.per_device_eval_batch_size
        return per_device_batch_size * max(1, self.n_gpu)

    @cached_property
    @torch_required
    def _setup_devices(self) -> Tuple["torch.device", int]:
        logger.info("PyTorch: setting up devices")
        if self.no_cuda:
            device = torch.device("cpu")
            n_gpu = 0
        elif is_tpu_available():
            device = xm.xla_device()
            n_gpu = 0
        elif self.local_rank == -1:
            # if n_gpu is > 1 we'll use nn.DataParallel.
            # If you only want to use a specific subset of GPUs use `CUDA_VISIBLE_DEVICES=0`
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            n_gpu = torch.cuda.device_count()
        else:
            # Here, we'll use torch.distributed.
            # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
            torch.distributed.init_process_group(backend="nccl")
            device = torch.device("cuda", self.local_rank)
            n_gpu = 1
        return device, n_gpu

    @property
    @torch_required
    def device(self) -> "torch.device":
        return self._setup_devices[0]

    @property
    @torch_required
    def n_gpu(self):
        return self._setup_devices[1]

    def to_json_string(self):
        """
        Serializes this instance to a JSON string.
        """
        return json.dumps(dataclasses.asdict(self), indent=2)

    def to_sanitized_dict(self) -> Dict[str, Any]:
        """
        Sanitized serialization to use with TensorBoard’s hparams
        """
        d = dataclasses.asdict(self)
        valid_types = [bool, int, float, str]
        if is_torch_available():
            valid_types.append(torch.Tensor)
        return {k: v if type(v) in valid_types else str(v) for k, v in d.items()}


Overwriting /kaggle/working/InfoBERT/ANLI/advtraining_args.py


In [12]:
%%writefile /kaggle/working/InfoBERT/ANLI/setup.sh


#!/usr/bin/env bash

function runexp {

export GLUE_DIR=anli_data
export TASK_NAME=${1}

custom=${2}   # Custom name
mname=${3}    # Model name
lr=${4}       # Learning rate for model parameters
bsize=${5}    # Batch size
seqlen=${6}    # Maximum sequence length
ts=${7}      # Number of training steps (counted as parameter updates)
ws=${8}      # Learning rate warm-up steps
seed=${9}    # Seed for randomness
wd=${10}      # Weight decay
beta=${11}    # regularizer coefficient
version=${12} # mi estimator version
hdp=${13}     # Hidden layer dropouts for ALBERT
adp=${14}     # Attention dropouts for ALBERT
alr=${15}      # Step size of gradient ascent
amag=${16}     # Magnitude of initial (adversarial?) perturbation
anorm=${17}    # Maximum norm of adversarial perturbation
asteps=${18}   # Number of gradient ascent steps for the adversary
alpha=${19}   # alpha for controlling local robust regularizer
cl=${20}   # lower threshold
ch=${21}   # higher threshold

if [ -z "${20}" ] ;then
    cl=0.5
fi

if [ -z "${21}" ] ;then
    ch=0.9
fi

expname=${custom}-${mname}-${TASK_NAME}-sl${seqlen}-lr${lr}-bs${bsize}-ts${ts}-ws${ws}-wd${wd}-seed${seed}-beta${beta}-alpha${alpha}--cl${cl}-ch${ch}-alr${alr}-amag${amag}-anm${anorm}-as${asteps}-hdp${hdp}-adp${adp}-version${version}

max=-1
for file in ${expname}/checkpoint-*
do
  fname=$(basename ${file})
  num=${fname:11}
  [[ $num -gt $max ]] && max=$num
done

if [ $max -eq -1 ] ; then
 echo "Train from stratch"
else
  FILE="${expname}/checkpoint-$max/eval_hist.bin"
  if [[ -f "$FILE" ]]; then
      echo "$FILE exists."
      mname="${expname}/checkpoint-$max" && echo "Resume Training from checkpoint $mname"
  else
      echo "$FILE does not exists."
      if [ $max -eq 100 ]; then
        max=-1
        echo "Train from stratch"
      else
        max=$(($max-100))
        FILE="${expname}/checkpoint-$max/eval_hist.bin"
        echo "use $FILE instead."
        mname="${expname}/checkpoint-$max" && echo "Resume Training from checkpoint $mname"
      fi
  fi
fi


port=$(($RANDOM + 1024))
echo "Master port: ${port}"
python -m torch.distributed.launch --nproc_per_node=1 --master_port ${port} ./run_anli.py \
  --model_name_or_path ${mname} \
  --task_name $TASK_NAME \
  --do_train  \
  --do_eval \
  --data_dir $GLUE_DIR \
  --max_seq_length ${seqlen} \
  --per_device_train_batch_size ${bsize} \
  --learning_rate ${lr} \
  --max_steps ${ts} \
  --warmup_steps ${ws} \
  --weight_decay ${wd} \
  --seed ${seed} \
  --beta ${beta} \
  --logging_dir ${expname} \
  --output_dir ${expname} \
  --version ${version}  --evaluate_during_training\
  --logging_steps 2000 --save_steps 2000 \
  --hidden_dropout_prob ${hdp} --attention_probs_dropout_prob ${adp}  --overwrite_output_dir  \
  --adv_lr ${alr} --adv_init_mag ${amag} --adv_max_norm ${anorm} --adv_steps ${asteps} --alpha ${alpha} \
   --cl ${cl} --ch ${ch}
}



function evalexp {
#export NCCL_DEBUG=INFO
#export NCCL_IB_CUDA_SUPPORT=0
#export NCCL_P2P_DISABLE=0
#export NCCL_IB_DISABLE=1
#export NCCL_NET_GDR_LEVEL=3
#export NCCL_NET_GDR_READ=0
#export NCCL_SHM_DISABLE=0

export GLUE_DIR=anli_data
export TASK_NAME="anli-full"

mname=${2}    # Model name
custom=${1}   # Custom name
lr=5e-3       # Learning rate for model parameters
bsize=32    # Batch size
seqlen=128    # Maximum sequence length
ts=0      # Number of training steps (counted as parameter updates)
ws=0      # Learning rate warm-up steps
seed=42    # Seed for randomness
wd=1e-5      # Weight decay
beta=0    # regularizer coefficient
version=3 # mi estimator version
hdp=0     # Hidden layer dropouts for ALBERT
adp=0     # Attention dropouts for ALBERT
alr=0      # Step size of gradient ascent
amag=0     # Magnitude of initial (adversarial?) perturbation
anorm=0    # Maximum norm of adversarial perturbation
asteps=1   # Number of gradient ascent steps for the adversary
alpha=0   # alpha for controlling local robust regularizer
cl=0   # lower threshold
ch=0   # higher threshold

expname=${custom}-load
port=$(($RANDOM + 1024))
echo "Master port: ${port}"

python -m torch.distributed.launch --nproc_per_node=1 --master_port ${port}  ./run_anli.py \
  --model_name_or_path ${mname} \
  --task_name $TASK_NAME \
  --do_eval \
  --data_dir $GLUE_DIR \
  --max_seq_length ${seqlen} \
  --per_device_train_batch_size ${bsize} \
  --learning_rate 3e-5 \
  --max_steps ${ts} \
  --warmup_steps ${ws} \
  --weight_decay ${wd} \
  --seed ${seed} \
  --beta ${beta} \
  --logging_dir ${expname} \
  --output_dir ${expname} \
  --version ${version} \
  --logging_steps 1000 --save_steps 1000 \
  --hidden_dropout_prob ${hdp} --attention_probs_dropout_prob ${adp}  --evaluate_during_training \
  --adv_lr ${alr} --adv_init_mag ${amag} --adv_max_norm ${anorm} --adv_steps ${asteps} --overwrite_output_dir \
  --alpha ${alpha} --cl ${cl} --ch ${ch}

tail -n +1 "${expname}"/eval_results*.txt > "${expname}"/eval_results.txt
cat "${expname}"/eval_results.txt
}


Overwriting /kaggle/working/InfoBERT/ANLI/setup.sh


In [13]:
!source activate newCondaEnvironment && source setup.sh && runexp anli-full infobert roberta-base 2e-5 32 128 -1 1000 42 1e-5 5e-3 6 0.1 0 4e-2 8e-2 0 3 0 0.5 0.9

setup.sh: line 46: [[: *: syntax error: operand expected (error token is "*")
Train from stratch
Master port: 11812
11/19/2024 19:45:18 - INFO - advtraining_args -   PyTorch: setting up devices
11/19/2024 19:45:18 - WARNING - __main__ -   Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: False
11/19/2024 19:45:18 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='infobert-roberta-base-anli-full-sl128-lr2e-5-bs32-ts-1-ws1000-wd1e-5-seed42-beta5e-3-alpha0--cl0.5-ch0.9-alr4e-2-amag8e-2-anm0-as3-hdp0.1-adp0-version6', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=2e-05, weight_decay=1e-05, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=1000, logging_dir='in